# Appunti sulla tesi *ottimizzazione degli algoritmi di ordinamento utilizzando AVX512*

## Leggiamo le istruzioni ASM di bubbleSort e selectionSort compilate con varie opzioni:
Per comodità utilizzo [godbolt](https://godbolt.org/) per leggere le istruzioni ASM

## BubbleSort:

In [ ]:
void bubbleSort(double * v, int n) {
    for(int i = n - 1 ; i >= 0 ; i--) {
        for(int j = 0 ; j < i ; j++) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
        }
    }
}

- senza parametri (standard -O0): utilizza i registri principali (rax,rdx,rcx) e traduce ~ 1:1 il codice in c, lungo circa 200 istruzioni
- -O1: analogo a prima, ma lungo circa 30 istruzioni
- -O2: come con `-O1` ma utilizza i registri xmm e l'istruzione `pshufd` per velocizzare le comparazioni, circa 23 istruzioni
- -O3: identico a `-O2`, nessuna differenza
- aggiungendo **#pragma GCC target("avx512f,avx512dq,avx512cd,avx512bw,avx512vl,avx512vbmi,avx512ifma,avx512pf,avx512er,avx5124fmaps,avx5124vnniw,avx512bitalg,avx512vp2intersect") #include <immintrin.h>** per cercare di spingere l'utilizzo di AVX512 i risultati non cambiano per `-O1, -O2, -O3`


# SelectionSort

In [ ]:
void selectionSort(double * v, int dim) {
    for(int i = 0 ; i < dim ; i++) {
        int idx = i;
        for(int j = i + 1 ; j < dim ; j++) {
            if(v[j] < v[idx]) {
                idx = j;
            }
        }
        swap(v[i],v[idx]);
    }
}

- Senza parametri (standard -O0): utilizza i registri principali e traduce ~1:1, ~80 istruzioni
- -O1: utilizza anche registri come r9/r10/r11, sempre 1:1, ~50 istruzioni
- -O2: come `-O1` ma ~30 istruzioni
- -O3: identico a `-O2`, nessuna differenza
- aggiungendo **#pragma GCC target("avx512f,avx512dq,avx512cd,avx512bw,avx512vl,avx512vbmi,avx512ifma,avx512pf,avx512er,avx5124fmaps,avx5124vnniw,avx512bitalg,avx512vp2intersect") #include <immintrin.h>** per cercare di spingere l'utilizzo di AVX512 i risultati non cambiano per `-O1, -O2, -O3`

# CountingSort
```
void countingSort(int * v, int n) {
    int a = v[0];
    int b = v[0];
    for(int i = 0 ; i < n ; i++) {
        a = std::max(a,v[i]);
        b = std::min(b,v[i]);
    }
    int counts[a-b+1];
    for(int i = 0 ; i < a-b+1 ; i++) {
        counts[i] = 0;
    }
    for(int i = 0 ; i < n ; i++) {
        counts[v[i]-b]++;
    }
    int idx = 0, i = 0;
    while(idx < a-b+1) {
        if(counts[idx] != 0) {
            v[i++] = idx+b;
            counts[idx]--;
        } else {
            idx++;
        }
    }
}
```
*inserito il limite per ogni variabile a 240000*
- Senza parametri (-O0): utilizza i registri principali e traduce ~1:1, ~150 istruzioni
- -O1: utilizza nache r8/r9, ~80 istruzioni
- -O2: utilizza anche r10+, ~90 istruzioni
- -O3: utilizza i registri xmm, ~170 istruzioni

## tabella tempi, utilizzata la libreria fatta da Sansone(?) "timer.cpp"

tempi presi in maniera non rigorosa, utilizzati per fare una prima scrematura, su un array di 1000 valori interi generati con funzione rand(), massimo INT_MAX, con 1000 prove

|parametri | bubbleSort | selectionSort | insertionSort | countingSort | 
|---|---|---|---|---|
| default (-O0) | 1125±283 | 534±195 | 757±230 |
| -O1 | 349±150 | 275±114 | 454±157 |
| -O2 | 1558±264 | 275±116 | 380±156 |
| -O3 | 1559±256 | 279±122 | 441±178 |
| -O0 + pragma | 1118±247 | 545±202 | 406±156 | 756±210 |
| -O1 + pragma| 337±135 | 276±114 | 90±41 | 469±152 |
| -O2 + pragma| 372±160 | 275±109 | 79±36 | 376±132 |
| -O3 + pragma| 370±156 | 276±118 | 76±34 |456±166 |

# Utilizzando le intrinsics
Come visto precedentemente il compilatore sfrutta i registri AVX solo per velocizzare le comparazioni tra 2 elementi, non sfruttando le potenzialità delle istruzioni SIMD.
Adesso scrivo il codice con le intrinsics SIMD prese da [Intel® Intrinsics Guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html)

# SelectionSort
Iniziamo dall'algoritmo che promette un miglioramento maggiore
Da ora in avanti tutti i test sono fatti con parametri -O1, -O2 e -O3 perchè -O0 dà sempre SIGSEGV alle versioni vettorizzate

Ho scritto due versioni del selectionSort ricercando le funzioni necessarie su [Intel® Intrinsics Guide](https://www.intel.com/content/www/us/en/docs/intrinsics-guide/index.html) per utilizzare istruzioni AVX512.

La prima versione è molto simile al selectionSort senza istruzioni SIMD ma fa paragoni di 8 variabili alla volta e se trova un nuovo minimo cerca elemento per elemento a quale indice si trova.

La seconda versione ha la stessa struttura, ma riduce il tempo di esecuzione evitando di fare un ciclo per trovare l'indice, ma utilizzando un algoritmo branchless che sfrutta istruzioni SIMD.

Per trovare gli indici esegue una maskz_abs (non ho trovato nessuna funzione che eseguisse soltato una maschera) lasciando nel registro *c* solo gli indici dei valori minimi correnti. A questo punto mi basta uno qualsiasi dei valori diversi da zero rimasti nel registro *c*, utilizzo quindi una reduce_max che restituisce il massimo.

Mi aspetto che esistano intrinsics più efficienti, ma al momento è comunque più che sufficiente.

In [ ]:
void selectionSortAVX512_v1(double * v, int dim) {
    __m512d arr;
    for(int i = 0 ; i < dim ; i++) {
        double minimum = v[i];
        double lastMin = minimum;
        int idx = i;
        int j = i;
        while(j <= dim - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            minimum = min(minimum,_mm512_reduce_min_pd(arr));
            if(minimum != lastMin) {
                lastMin = minimum;
                for(int k = 0 ; k < 8 ; k++) {
                    if(v[j+k] == minimum) {
                        idx = j+k;
                        break;
                    }
                }
            }
            j += 8;
        }
        // use regular code to check the last values with index not multiple of 8
        while(j < dim) {
            if(v[j] < v[idx]) {
                idx = j;
            }
            j++;
        }
        swap(v[i],v[idx]);
    }
}

In [ ]:
void selectionSortAVX512_v2(double * v, int dim) {
    __m512d arr,min_vect;
    long idxs_arr[8] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_load_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = 0 ; i < dim ; i++) {
        double minimum = v[i];
        double last = minimum;
        int idx = i;
        int j = i;
        while(j <= dim - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            minimum = min(minimum,_mm512_reduce_min_pd(arr));
            if(minimum != last) {
                last = minimum;
                min_vect = _mm512_set1_pd(minimum);
                mask = _mm512_cmpeq_pd_mask(arr,min_vect);
                c = _mm512_maskz_abs_epi64(mask,idxs_vect);
                idx = j + _mm512_reduce_max_epi64(c);
            }
            j+=8;
        }
        // use regular code to check the last values with index not multiple of 8
        while(j < dim) {
            if(v[j] < v[idx]) {
                idx = j;
            }
            j++;
        }
        swap(v[i],v[idx]);
    }
}

### Statistiche `DA RISCRIVERE`
Ho testato i vari algoritmi utilizzando *-O3*, *-O2*, *-O1* ; per ogni parametro ho testato array di lunghezza 1000, 10000 e 100000 ; per ognuno di questi ho fatto 100 simulazioni, ogni volta **i numeri inseriti nell'array sono in ordine decrescente**.

Di seguito le mie considerazioni.

- Come da aspettative ogni distribuzione dei tempi è molto ravvicinata, con una breve e rarefatta coda verso i tempi maggiori
- Indipendentemente dalla lunghezza degli array i rapporti tra i vari algoritmi sono gli stessi, ma il grafico più esplicativo è quello in cui gli array hanno dimensione 10000.
- L'algoritmo v0 (senza intrinsics ma ottimizzata solo dal compilatore) più veloce è utilizzando la flag *-O1*, che è circa il 22% più veloce dell'algoritmo *v0 -O3*.
- L'algoritmo *v1* ha minime differenze tra *-O3*, *-O2* e *-O1*.
- L'algoritmo *v2* invece ha differenze minime solo tra *-O2* e *-O1*.
- Prendendo come base l'algoritmo *v0 -O1* (e considerando solo le mediane):
    - L'algoritmo *v1 -O3* è il 10% più veloce
    - L'algoritmo *v2 -O2* è il 27% più veloce

Testando tutte le versioni degli algoritmi con array i cui dati sono generati casualmente (attraverso la `rand()` dopo una `srand(time(NULL))`) la principale differenza è che l'algoritmo *v1* si avvicina molto a quello *v2*.

Prendendo ancora come base l'algoritmo *v0 -O1* (e considerando solo le mediane):
- L'algoritmo *v1 -O3* è il 51% più veloce
- L'algoritmo *v2 -O1* è il 54% più veloce

# Considerazioni
Premessa

Ho testato le 3 versioni di *selectionSort* con array di dimensione 10,100,1000,10000 e 100000. Per ogni dimensione ho compilato sia con *-O1*, *-O2*, *-O3* (per tutti ho utilizzato *-march=znver5*). Per ognuno ho provato sia con la versione 0(senza SIMD),1 e 2 dell'algoritmo. Ogniuno è stato testato con array in ordine casuale, crescente e decrescente. Per ogni combinazione ho eseguito 20 volte per avere un sample size minimo.

## Osservazioni 
- La flag *-O1* è quella che rende la versione *v0* $\textrm{consistentemente}^{1}$ più rapido
- La flag *-O3* è quella che rende la versione *v1* $\textrm{consistentemente}^{1}$ (ma minimamente) più lento
- Non ci sono sostanziali differenze tra *-O1*, *-O2* e *-O3* utilizzando la versione *v2*
- Per array di dimensione 10 i valori sono troppo ravvicinati per ricavarne considerazioni
- Per array di dimensione 100 si inizia a notare che il caso in cui i dati sono generati in ordine decrescente impiegano $\textrm{consistentemente}^{2}$ meno tempo di quelli in cui sono inseriti in ordine casuale, ma non si apprezza differenze tra le versioni dell'algoritmo
- A partire da array di dimensione 1000 si nota che qualsiasi versione risulta consistentemente più rapida se eseguita su array ordinati in modo decrescente che in modo casuale (ipotizzo sia dovuto alla branch prediction del processore)
- Per array di dimensione 1000  notiamo che *v1* è il 13,6%$^{3}$ più veloce di *v0*, mentre *v2* lo è il 31,6%$^{3}$.

Note:
1. Per ogni lunghezza testata e per ogni ordinamento iniziale dell'array
2. Per ogni lunghezza testata
3. compilando tutto con flag *-O1* e utilizzando la mediana dei tempi impiegati per ordinare un array di numeri generati casualmente


# Altre versioni
Ho deciso di scrivere le mie versioni dell'algoritmo prima di fare di cerche per evitare di essere influenzato da algoritmi altrui.
Cercando su internet non si trovano versioni di facile lettura, ma utilizzando i principali tool di assistenza alla programmazione possimamo ottenere alcuni spunti.
- *ChatGPT*, anche dopo alcuni $\textrm{solleciti}^{1}$, non restituisce un codice che esegue come richiesto.
- *Microsoft copilot* restituisce un codice che esegue come richiesto e che, sfruttando instrinsics di cui non ero a conoscenza.

Si osserva che la versione di *copilot* non necessita di un ciclo per gli ultimi massimo 7 valori, questo è dovuto alla prima condizione dell'if più interno.

Note:
1. Per solleciti sis intende messaggi in cui si precisa quale parte del codice correggere

In [ ]:
void selectionSortAVX512_copilot(double* arr, int n) {
    __m512d current_min_vals;
    __mmask8 mask;
    __m512d current_vals;
    for (int i = 0; i < n - 1; ++i) {
        int min_idx = i;
        double min_val = arr[i];

        for (int j = i + 1; j < n; j += 8) {
            __m512d current_vals = _mm512_loadu_pd(&arr[j]);
            __m512d current_min_vals = _mm512_set1_pd(min_val);
            __mmask8 mask = _mm512_cmplt_pd_mask(current_vals, current_min_vals);
            if (mask) {
                // Extract the new minimum value from current_vals
                for (int k = 0; k < 8; ++k) {
                    if (j + k < n && (mask & (1 << k))) {
                        if (arr[j + k] < min_val) {
                            min_val = arr[j + k];
                            min_idx = j + k;
                        }
                    }
                }
            }
        }

        // Swap the found minimum element with the first element
        if (min_idx != i) {
            std::swap(arr[i], arr[min_idx]);
        }
    }
}

In [ ]:
// per completezza
void selectionSortAVX512_ChatGPT(double* arr, size_t size) {
    for (size_t i = 0; i < size; ++i) {
        size_t min_index = i;
        double min_val = arr[i];

        // Find the minimum in the remaining array using AVX-512
        for (size_t j = i; j + 8 <= size; j += 8) {
            // Load 8 elements into an AVX-512 register
            __m512d reg = _mm512_loadu_pd(&arr[j]);

            // Compare and find the minimum value in the register
            __mmask8 cmp_mask = _mm512_cmp_pd_mask(reg, _mm512_set1_pd(min_val), _CMP_LT_OQ);

            // Update min_val and min_index directly using intrinsics
            if (cmp_mask) {
                // prende il primo, non funziona
                int first_true = _tzcnt_u32(cmp_mask); // Get the index of the first true comparison
                min_val = arr[j + first_true];
                min_index = j + first_true;
            }
        }

        // Check any remaining elements not fitting in an AVX-512 register
        for (size_t j = (size / 8) * 8; j < size; ++j) {
            if (arr[j] < min_val) {
                min_val = arr[j];
                min_index = j;
            }
        }

        // Swap the found minimum with the current element
        if (min_index != i) {
            std::swap(arr[i], arr[min_index]);
        }
    }
}

## Osservazioni
La versione *copilot* è sostanzialmente equivalente alla *v2*$^{1}$ per array di dimensione 1000, mentre al crescere della dimensione degli array aumenta il distacco a suo favore. Questo è dovuto al fatto che il codice tra il secondo for e l'if successivo è molto più efficiente dell'analoga parte di *v2*. Tuttavia la parte interna al suddetto if è più rapida nella versione *v2*.

Unendo le due versioni otteniamo la versione *v5*.

Note:
1. per array non ordinati

In [ ]:
void selectionSortAVX512_v5(double * v, int dim) {
    __m512d arr,min_vect;
    long idxs_arr[8] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_load_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = 0 ; i < dim ; i++) {
        double minimum = v[i];
        double last = minimum;
        int idx = i;
        int j = i + 1;
        min_vect = _mm512_set1_pd(minimum);
        while(j < dim -8) {
            arr = _mm512_loadu_pd(&v[j]);
            mask = _mm512_cmplt_pd_mask(arr, min_vect);
            if(mask) {
                minimum = _mm512_reduce_min_pd(arr);
                min_vect = _mm512_set1_pd(minimum);
                mask = _mm512_cmpeq_pd_mask(arr,min_vect);
                c = _mm512_maskz_abs_epi64(mask,idxs_vect);
                idx = j + _mm512_reduce_max_epi64(c);
                min_vect = _mm512_set1_pd(minimum);
            }
            j+=8;
        }
        while(j < dim) {
            if(v[j] < v[idx]) {
                idx = j;
            }
            j++;
        }
        swap(v[i],v[idx]);
    }
}

Con array di lunghezza 1000 questa versione compilata con la flag *-O1* risulta la più lenta con uno scarto del 6.8%, tuttavia all'aumentare della lunghezza la differenza diventa sempre più insignificante.

Su array di dimensione 1000 questa versione è il 29.2%$^{2}$ più veloce delle versioni *v2* e *copilot*, mentre è il 51.7% più veloce della versione *v0*

All'aumentare della dimensione la versione *copilot* si avvicina sempre di più, perché la parte di codice che le distingue viene eseguita meno $\textrm{volte}^{3}$

Note:
1. Su array con valori casuali
2. Statistica calcolata su 100 esecuzioni con array di elementi casuali casuali
3. Possiamo notarlo anche con la differenza tra le versioni *v1* e *v2*

eseguendo un test estensivo ho notato una estrema differenza tra le statistiche degli algoritmi compilati con la flag *-O1* e gli altri(quasi il doppio nel tempo di esecuzione), quindi ho deciso di eseguire nuovamente i test ma lasciando al computer più spazio per aspirare l'aria, sperando che sia sufficiente. Altrimenti sarò costtretto ad eseguire un carico molto elevato prima in modo da avere il processore in thermal protection durante tutti i test.
pare abbia trovato un equilibrio intorno ai 65/66 gradi

# bubbleSort
Analogamente al selectionSort ho implementato 2 versioni del bubbleSort più una in cui il ciclo interno è branchless

In [ ]:
void bubbleSortAVX512_v1(double * v, int dim) {
    __m512d arr;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        double maximum;
        int j = 0;
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            maximum = _mm512_reduce_max_pd(arr);
            if(maximum != v[j+7]) {
                for(int k = 0 ; k < 8 ; k++) {
                    if(v[j+k] == maximum) {
                        std::swap(v[j+7],v[j+k]);
                        break;
                    }
                }
            }
            j += 7;
        }
        while(j <= i - 1) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
            j++;
        }
    } 
    t.stop();
}

In [ ]:
void bubbleSortAVX512_v2(double * v, int dim) {
    __m512d arr,max_vect;
    long idxs_arr[16] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_load_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        double maximum;
        int j = 0;
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            maximum = _mm512_reduce_max_pd(arr);
            if(maximum != v[j+7]) {
                max_vect = _mm512_set1_pd(maximum);
                mask = _mm512_cmpeq_pd_mask(arr,max_vect);
                c = _mm512_maskz_abs_epi64(mask,idxs_vect);
                int idx = j + _mm512_reduce_max_epi64(c);
                std::swap(v[idx],v[j+7]);
            }
            j += 7;
        }
        while(j <= i - 1) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
            j++;
        }
    }
}

In [ ]:
void bubbleSortAVX512_v3(double * v, int dim) {
    __m512d arr,max_vect;
    long idxs_arr[16] = {0,1,2,3,4,5,6,7};
    __m512i c,idxs_vect = _mm512_load_epi64(idxs_arr);
    __mmask8 mask;
    for(int i = dim - 1 ; i >= 0 ; i--) {
        double maximum;
        int j = 0;
        while(j < i - 8) {
            arr = _mm512_loadu_pd(&v[j]);
            maximum = _mm512_reduce_max_pd(arr);
            max_vect = _mm512_set1_pd(maximum);
            mask = _mm512_cmpeq_pd_mask(arr,max_vect);
            c = _mm512_maskz_abs_epi64(mask,idxs_vect);
            int idx = j + _mm512_reduce_max_epi64(c);
            std::swap(v[idx],v[j+7]);
            j += 7;
        }
        while(j <= i - 1) {
            if(v[j] > v[j+1]) {
                std::swap(v[j],v[j+1]);
            }
            j++;
        }
    }
}

Ho deciso di implementare il bubbleSort invertendo l'elemento di valore maggiore con quello nel registro con indice più alto in ogni passata del ciclo interno.
Analogamente al selectionSort ho scritto una prima versione che utilizza un ciclo for e una che utilizza le istruzioni SIMD per trovare l'indice del valore massimo, mentre la terza sposta a priori il massimo all'indice massimo del vettore.

Considerazioni:
- 